In [76]:
import io
import csv
import pandas as pd
from urllib import request

response = request.urlopen('https://dq-content.s3.amazonaws.com/251/storm_data.csv')
reader = csv.reader(io.TextIOWrapper(response))

next(reader)

## Lets do a check of the data, to get a sense of length and values for what
## data type is most appropriate. Check all data values using a dictionary
## and Pandas Series.

check = []
for line in reader:
    check.append(line[13])
    
data = pd.Series(check)

data.value_counts()


0.806226    1350
0.5         1262
0.707107    1204
0.921954    1175
1.118034    1101
1            925
0.509902     892
0.608276     825
0.583095     817
1.140175     807
0.640312     806
0.360555     799
0.538516     794
1.004988     785
1.204159     774
0.316228     767
0.447214     744
0.412311     743
0.223607     739
0.72111      739
0.67082      720
0.728011     704
0.905539     685
0.632456     685
0.761577     677
0.781025     676
1.30384      660
0.824621     658
1.019804     654
1.044031     650
            ... 
7.433034       1
4.879549       1
6.280127       1
6.216912       1
6.870953       1
6.907966       1
4.414748       1
4.998          1
4.410215       1
6.685058       1
4.664762       1
5.95483        1
5.434151       1
5.500909       1
5.53263        1
4.079216       1
5.869412       1
5.824088       1
6.168468       1
3.984972       1
7.106335       1
7.338937       1
6.488451       1
4.854894       1
1.00005        1
7.119691       1
4.580393       1
11.18034      

In [105]:
## Connect to local postgres database and create our storm data table

import psycopg2 as psy

conn = psy.connect(dbname="postgres", password="postgres")

cur = conn.cursor()

cur.execute("""CREATE TABLE storm_data (
           fid INTEGER PRIMARY KEY,
           year SMALLINT,
           month SMALLINT,
           day SMALLINT,
           ad_time VARCHAR(5),
           btid SMALLINT,
           name VARCHAR(20),
           lat DECIMAL(10,8),
           long DECIMAL(11,8),
           wind_kts SMALLINT,
           pressure SMALLINT,
           cat VARCHAR(2),
           basin VARCHAR(15),
           shape_leng REAL
           )""")
conn.commit()

In [106]:
## Let's check that our table was successfully created.

cur.execute("SELECT * FROM storm_data")
print(cur.description)

(Column(name='fid', type_code=23, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None), Column(name='year', type_code=21, display_size=None, internal_size=2, precision=None, scale=None, null_ok=None), Column(name='month', type_code=21, display_size=None, internal_size=2, precision=None, scale=None, null_ok=None), Column(name='day', type_code=21, display_size=None, internal_size=2, precision=None, scale=None, null_ok=None), Column(name='ad_time', type_code=1043, display_size=None, internal_size=5, precision=None, scale=None, null_ok=None), Column(name='btid', type_code=21, display_size=None, internal_size=2, precision=None, scale=None, null_ok=None), Column(name='name', type_code=1043, display_size=None, internal_size=20, precision=None, scale=None, null_ok=None), Column(name='lat', type_code=1700, display_size=None, internal_size=10, precision=10, scale=8, null_ok=None), Column(name='long', type_code=1700, display_size=None, internal_size=11, precision=11, scal

In [107]:
## Let's load our data with the reader object we created above.


response = request.urlopen('https://dq-content.s3.amazonaws.com/251/storm_data.csv')
reader = csv.reader(io.TextIOWrapper(response))

next(reader)

for row in reader:
    cur.execute(
            """INSERT INTO storm_data VALUES (%s, %s, %s, %s, %s, %s, %s, %s,
            %s, %s, %s, %s, %s, %s)
            """,
            row
        )

conn.commit()



KeyboardInterrupt: 

In [ ]:
## Let's add a storm date column so we have the year, month and date in one column.

cur.execute(""" ALTER TABLE storm_data
                ADD COLUMN date DATE 
            """)

cur.execute(""" UPDATE storm_data
                SET date = to_date(year || '-' || month || '-' || day, 'YYYY-MM-DD')
            """)

conn.commit()

In [ ]:
## Now let's create a user that can read, insert, and update data but not delete it.

cur.execute("CREATE USER data_analyst PASSWORD 'analyst123'")
cur.execute("REVOKE ALL ON storm_data FROM data_analyst")
cur.execute("GRANT SELECT ON storm_data TO data_analyst")
cur.execute("GRANT INSERT ON storm_data TO data_analyst")
cur.execute("GRANT UPDATE ON storm_data TO data_analyst")

conn.commit()

